In [1]:
import numpy as np
from qiskit import Aer
from qiskit.transpiler import PassManager

from qiskit_aqua import Operator, QuantumInstance
from qiskit_aqua.algorithms.adaptive import VQE
from qiskit_aqua.algorithms.classical import ExactEigensolver
from qiskit_aqua.components.optimizers import L_BFGS_B
from qiskit_aqua.components.variational_forms import RY

from qiskit_chemistry import FermionicOperator
from qiskit_chemistry.drivers import PyQuanteDriver, UnitsType, BasisType

In [2]:
# using driver to get fermionic Hamiltonian
# PyQuante example
driver = PyQuanteDriver(atoms='H .0 .0 .0; H .0 .0 0.735', units=UnitsType.ANGSTROM,
                        charge=0, multiplicity=1, basis=BasisType.BSTO3G)
molecule = driver.run()
h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals

In [3]:
# convert from fermionic hamiltonian to qubit hamiltonian
ferOp = FermionicOperator(h1=h1, h2=h2)
qubitOp_jw = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp_pa = ferOp.mapping(map_type='PARITY', threshold=0.00000001)
qubitOp_bi = ferOp.mapping(map_type='BRAVYI_KITAEV', threshold=0.00000001)

In [4]:
# print out qubit hamiltonian in Pauli terms and exact solution

qubitOp_jw.to_matrix()
qubitOp_jw.chop(10**-10)

print(qubitOp_jw.print_operators())

# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = ExactEigensolver(qubitOp_jw, k=1)
ret = exact_eigensolver.run()
print('The exact ground state energy is: {}'.format(ret['energy']))    

IIII	(-0.8105479862760991+0j)
IIIZ	(0.17218394273085635+0j)
IIZI	(-0.22575350251540605+0j)
IIZZ	(0.12091263358559995+0j)
IZII	(0.17218394273085635+0j)
IZIZ	(0.16892754048859007+0j)
IZZI	(0.16614543338049342+0j)
IZZZ	(-8.326672684688674e-17+0j)
XXXX	(0.045232799794893426+0j)
XXYY	(0.045232799794893426+0j)
YYXX	(0.045232799794893426+0j)
YYYY	(0.045232799794893426+0j)
ZIII	(-0.2257535025154061+0j)
ZIIZ	(0.16614543338049342+0j)
ZIZI	(0.17464343142442207+0j)
ZZII	(0.12091263358559991+0j)
ZZIZ	(-2.42861286636753e-17+0j)
ZZZI	(-6.938893903907228e-17+0j)
ZZZZ	(-3.122502256758253e-17+0j)

The exact ground state energy is: -1.8572750766378763


In [5]:
# setup VQE 
# setup optimizer, use L_BFGS_B optimizer for example
lbfgs = L_BFGS_B(maxfun=1000, factr=10, iprint=10)

# setup variational form generator (generate trial circuits for VQE)
var_form = RY(qubitOp_jw.num_qubits, 5, entangler_map = {0: [1], 1:[2], 2:[3]})

# setup VQE with operator, variational form, and optimizer
vqe_algorithm = VQE(qubitOp_jw, var_form, lbfgs, 'matrix')

backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, pass_manager=PassManager())

results = vqe_algorithm.run(quantum_instance)

print("Minimum value: {}".format(results['eigvals'][0]))
print("Parameters: {}".format(results['opt_params']))

Minimum value: -1.857275076588138
Parameters: [ 1.72459655  1.92029571 -0.34982926 -1.49153618  2.26170073  0.1302224
  0.72586921  0.29764644 -1.401497    1.49290088 -3.09131598  0.1647657
 -2.69893629 -1.46110898 -1.99677374  1.92441472 -0.94314616  2.69400524
  2.75985138  0.16260948 -2.3450682   1.46493868  1.22492389  3.03079433]
